In [1]:
%load_ext autoreload
%autoreload 2
import pytorch_lightning as pl
import pytorch_lightning.callbacks as pl_callbacks
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import eq


from eq.data import Catalog, InMemoryDataset, Sequence, default_catalogs_dir

In [2]:
anss = eq.catalogs.ANSS_MultiCatalog(num_sequences=5)

Loading existing catalog from /home/gcl/RA/jonahk/recast/data/ANSS_MultiCatalog.


In [3]:
len(anss)

TypeError: object of type 'ANSS_MultiCatalog' has no len()

In [6]:

dl_train = anss.train.get_dataloader( batch_size=200, shuffle=False)
dl_val = anss.val.get_dataloader(batch_size=200, shuffle=False)
dl_test = anss.test.get_dataloader(batch_size=200, shuffle=False)

In [7]:
batch = next(iter(dl_train))
batch

Batch(
  inter_times: [3, 140],
  arrival_times: [3, 140],
  t_start: [3],
  t_end: [3],
  t_nll_start: [3],
  mask: [3, 140],
  start_idx: [3],
  end_idx: [3],
  mag: [3, 140],
  extra_feat: [3, 140, 1],
  mag_bounds: [3, 2],
  mag_nll_bounds: [3, 2]
)

In [10]:
model = eq.models.RecurrentTPP(
    num_extra_features=1, # the number of extra features
    tau_mean=(
        len(anss.train[0])/(anss.train[0].t_end-anss.train[0].t_start)
    ), # the mean of the inter-event time distribution
)

In [11]:
    # ModelCheckpoints saves the model with the best validation loss
checkpoint = pl_callbacks.ModelCheckpoint(monitor="total_val_loss")

    # EarlyStopping stops training if the validation loss doesn't improve by more than 1e-3 for 20 epochs
early_stopping = pl_callbacks.EarlyStopping(monitor="total_val_loss", patience=10, min_delta=1e-5)

    # RichProgressBar adds a nice and more functional progress bar
progress_bar = pl_callbacks.RichProgressBar()

    # Trainer set up training and validation loops with previous specs
trainer = pl.Trainer(devices=1,max_epochs=10000, callbacks=[checkpoint, early_stopping],log_every_n_steps=1)

trainer.fit(model, dl_train, dl_val)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name          | Type    | Params
------------------------------------------
0 | hypernet_time | Linear  | 3.2 K 
1 | hypernet_mag  | Linear  | 33    
2 | rnn           | GRU     | 3.6 K 
3 | dropout       | Dropout | 0     
------------------------------------------
6.8 K     Trainable params
0         Non-trainable params
6.8 K     Total params
0.027     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]